# レポート課題第9回

'todoufuken.csv'のデータに対して「最短距離法」と「ウォード法」の二種類の階層クラスタリングを行い，クラスタリング結果の違いに対する考察を行え。

Googleドライブのマウントとフォルダの移動

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/09/
# 現在のフォルダの中身を表示
%ls

必要ライブラリをインポート。  
日本語フォントもダウンロードとインポート。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install japanize-matplotlib
import japanize_matplotlib 

`todoufuken.csv`を読み込む。  
このデータは各都道府県の代表駅の東経と北緯を記録したデータである。  
東経と北緯のデータをXとして，都道府県の位置をクラスタリングする。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('todoufuken.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:,'東経':'北緯'].to_numpy()

# 各サンプルの都道府県をサンプルの名前とする。
labels = csv_data.loc[:,'都道府県'].to_list()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

データの標準化とプロットを行う。

In [ ]:
# データの標準化
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# 二次元データのプロット
plt.figure(figsize=(10,10))
plt.scatter(X_norm[:,0], X_norm[:,1], color='k')
plt.xlabel('東経（標準化済み）')
plt.ylabel('北緯（標準化済み）')

# 各サンプルの名前(ラベル)を付ける
for n in range(num_samples):
  label = str(labels[n])
  plt.annotate(label, (X_norm[n,0], X_norm[n,1]))

plt.show()

## <font color=red> 小問1: このデータに対して最短距離法を適用し，挙動を確認せよ。</font> 

## <font color=red>小問2: ウォード法による階層クラスタリングを実装し，その挙動を確認せよ。クラス分割数$K=6$のときのクラスタリング結果を図示せよ。</font>

ヒント  
* `get_shortest_distance`関数の要領で，ウォード法による距離を返す関数 `get_ward_distance`を作成する。
* クラス統合を繰り返すループ内にて，`dist = get_shortest_distance(class_list[i], class_list[j])`を`dist = get_ward_distance(class_list[i], class_list[j])`に変更する。
* [デバッグ情報] sklearnライブラリのAgglomerativeClusteringモジュールを使って実装した場合の結果を以下に示す。自分で実装した結果が合っているかの確認に利用してもらいたい。**課題はsklearnライブラリを使用せずに実装すること！**

In [ ]:
from sklearn.cluster import AgglomerativeClustering
model = AgglomerativeClustering(affinity='euclidean',linkage='ward', n_clusters=6)

model = model.fit(X_norm)


colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,10))
# セントロイドを"x"印でプロット 
for k in range(6):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  plt.scatter(X_norm[model.labels_==k,0], X_norm[model.labels_==k,1], color=c)
plt.xlabel('東経（標準化済み）')
plt.ylabel('北緯（標準化済み）')

# 各サンプルの名前(ラベル)を付ける
for n in range(num_samples):
  label = str(labels[n])
  plt.annotate(label, (X_norm[n,0], X_norm[n,1]))

plt.show()

## <font color=red>小問3: 最短距離法とウォード法の挙動を見比べ，どういった点が異なるか，また何故そのような違いが出るのかについて考察せよ。</font>  
【補足】  
階層型クラスタリングの様子をアニメーションにする場合，都道府県名のラベルは表示させないようにしてください。  
つまり，以下のコードは不要です．  
```
# 各サンプルの名前(ラベル)を付ける
for n in range(num_samples):
  label = str(labels[n])
  plt.annotate(label, (X_norm[n,0], X_norm[n,1]))
```

（`09_01_hierarchical_clustering.ipynb` からアニメーションを作成する部分をそのままコピペすれば，都道府県は表示されません。）  
都道府県名を表示させた状態でアニメーションを作成した場合，`convert_report.ipynb`を使ってHTMLファイル化した際に容量オーバーとなってアニメーションが表示されなくなる不具合があるためです。